# Chess-Bot

## Setup

In [2]:
%load_ext autoreload
%autoreload 2

import chessbot as cb

## Environment

In [3]:
cb.environment.create()

Dataset archive file found at /home/wumphlett/personal/Chess-Bot/data/CCRL-4040.[1505357].pgn.7z
Dataset file found at /home/wumphlett/personal/Chess-Bot/data/CCRL-4040.[1505357].pgn
Dataset pkl files found at /home/wumphlett/personal/Chess-Bot/data
Dataset val pkl files found at /home/wumphlett/personal/Chess-Bot/data
Done


## Pos2Vec

In [3]:
train, val = cb.dataset.pos2vec_dataset()

In [4]:
pos2vec = cb.model.train_pos2vec(train, val)

Training layer 1/4
Epoch 1/200
40000/40000 [==============================] - 69s 2ms/step - loss: 0.3097 - val_loss: 0.1194 - lr: 0.0050
Epoch 2/200
40000/40000 [==============================] - 70s 2ms/step - loss: 0.1073 - val_loss: 0.1005 - lr: 0.0049
Epoch 3/200
40000/40000 [==============================] - 71s 2ms/step - loss: 0.0972 - val_loss: 0.0944 - lr: 0.0048
Epoch 4/200
40000/40000 [==============================] - 70s 2ms/step - loss: 0.0921 - val_loss: 0.0899 - lr: 0.0047
Epoch 5/200
40000/40000 [==============================] - 70s 2ms/step - loss: 0.0882 - val_loss: 0.0865 - lr: 0.0046
Epoch 6/200
40000/40000 [==============================] - 70s 2ms/step - loss: 0.0851 - val_loss: 0.0839 - lr: 0.0045
Epoch 7/200
40000/40000 [==============================] - 72s 2ms/step - loss: 0.0828 - val_loss: 0.0818 - lr: 0.0044
Epoch 8/200
40000/40000 [==============================] - 69s 2ms/step - loss: 0.0810 - val_loss: 0.0801 - lr: 0.0043
Epoch 9/200
40000/40000 [====

40000/40000 [==============================] - 69s 2ms/step - loss: 0.0504 - val_loss: 0.0504 - lr: 0.0013
Epoch 70/200
40000/40000 [==============================] - 70s 2ms/step - loss: 0.0503 - val_loss: 0.0502 - lr: 0.0012
Epoch 71/200
40000/40000 [==============================] - 69s 2ms/step - loss: 0.0502 - val_loss: 0.0501 - lr: 0.0012
Epoch 72/200
40000/40000 [==============================] - 69s 2ms/step - loss: 0.0500 - val_loss: 0.0500 - lr: 0.0012
Epoch 73/200
40000/40000 [==============================] - 68s 2ms/step - loss: 0.0499 - val_loss: 0.0498 - lr: 0.0012
Epoch 74/200
40000/40000 [==============================] - 71s 2ms/step - loss: 0.0498 - val_loss: 0.0497 - lr: 0.0011
Epoch 75/200
40000/40000 [==============================] - 70s 2ms/step - loss: 0.0497 - val_loss: 0.0496 - lr: 0.0011
Epoch 76/200
40000/40000 [==============================] - 69s 2ms/step - loss: 0.0495 - val_loss: 0.0495 - lr: 0.0011
Epoch 77/200
40000/40000 [===========================

40000/40000 [==============================] - 69s 2ms/step - loss: 0.0460 - val_loss: 0.0459 - lr: 3.3363e-04
Epoch 136/200
40000/40000 [==============================] - 69s 2ms/step - loss: 0.0459 - val_loss: 0.0459 - lr: 3.2696e-04
Epoch 137/200
40000/40000 [==============================] - 69s 2ms/step - loss: 0.0459 - val_loss: 0.0459 - lr: 3.2042e-04
Epoch 138/200
40000/40000 [==============================] - 69s 2ms/step - loss: 0.0459 - val_loss: 0.0459 - lr: 3.1401e-04
Epoch 139/200
40000/40000 [==============================] - 68s 2ms/step - loss: 0.0459 - val_loss: 0.0459 - lr: 3.0773e-04
Epoch 140/200
40000/40000 [==============================] - 69s 2ms/step - loss: 0.0458 - val_loss: 0.0458 - lr: 3.0157e-04
Epoch 141/200
40000/40000 [==============================] - 69s 2ms/step - loss: 0.0458 - val_loss: 0.0458 - lr: 2.9554e-04
Epoch 142/200
40000/40000 [==============================] - 68s 2ms/step - loss: 0.0458 - val_loss: 0.0458 - lr: 2.8963e-04
Epoch 143/200


40000/40000 [==============================] - 68s 2ms/step - loss: 0.0453 - val_loss: 0.0453 - lr: 8.9734e-05
Training layer 2/4
Epoch 1/200
40000/40000 [==============================] - 71s 2ms/step - loss: 0.0785 - val_loss: 0.0704 - lr: 0.0050
Epoch 2/200
40000/40000 [==============================] - 72s 2ms/step - loss: 0.0670 - val_loss: 0.0643 - lr: 0.0049
Epoch 3/200
40000/40000 [==============================] - 70s 2ms/step - loss: 0.0624 - val_loss: 0.0608 - lr: 0.0048
Epoch 4/200
40000/40000 [==============================] - 70s 2ms/step - loss: 0.0595 - val_loss: 0.0584 - lr: 0.0047
Epoch 5/200
40000/40000 [==============================] - 70s 2ms/step - loss: 0.0575 - val_loss: 0.0567 - lr: 0.0046
Epoch 6/200
40000/40000 [==============================] - 72s 2ms/step - loss: 0.0559 - val_loss: 0.0553 - lr: 0.0045
Epoch 7/200
40000/40000 [==============================] - 71s 2ms/step - loss: 0.0547 - val_loss: 0.0541 - lr: 0.0044
Epoch 8/200
40000/40000 [============

40000/40000 [==============================] - 71s 2ms/step - loss: 0.0409 - val_loss: 0.0409 - lr: 0.0013
Epoch 69/200
40000/40000 [==============================] - 72s 2ms/step - loss: 0.0409 - val_loss: 0.0409 - lr: 0.0013
Epoch 70/200
40000/40000 [==============================] - 70s 2ms/step - loss: 0.0408 - val_loss: 0.0408 - lr: 0.0012
Epoch 71/200
40000/40000 [==============================] - 70s 2ms/step - loss: 0.0408 - val_loss: 0.0408 - lr: 0.0012
Epoch 72/200
40000/40000 [==============================] - 71s 2ms/step - loss: 0.0408 - val_loss: 0.0407 - lr: 0.0012
Epoch 73/200
40000/40000 [==============================] - 70s 2ms/step - loss: 0.0407 - val_loss: 0.0407 - lr: 0.0012
Epoch 74/200
40000/40000 [==============================] - 70s 2ms/step - loss: 0.0407 - val_loss: 0.0406 - lr: 0.0011
Epoch 75/200
40000/40000 [==============================] - 70s 2ms/step - loss: 0.0406 - val_loss: 0.0406 - lr: 0.0011
Epoch 76/200
40000/40000 [===========================

...

40000/40000 [==============================] - 81s 2ms/step - loss: 0.0400 - val_loss: 0.0400 - lr: 3.4739e-04
Epoch 134/200
40000/40000 [==============================] - 77s 2ms/step - loss: 0.0400 - val_loss: 0.0400 - lr: 3.4044e-04
Epoch 135/200
40000/40000 [==============================] - 78s 2ms/step - loss: 0.0400 - val_loss: 0.0400 - lr: 3.3363e-04
Epoch 136/200
40000/40000 [==============================] - 82s 2ms/step - loss: 0.0400 - val_loss: 0.0400 - lr: 3.2696e-04
Epoch 137/200
40000/40000 [==============================] - 77s 2ms/step - loss: 0.0400 - val_loss: 0.0400 - lr: 3.2042e-04
Epoch 138/200
40000/40000 [==============================] - 77s 2ms/step - loss: 0.0400 - val_loss: 0.0400 - lr: 3.1401e-04
Epoch 139/200
40000/40000 [==============================] - 77s 2ms/step - loss: 0.0400 - val_loss: 0.0400 - lr: 3.0773e-04
Epoch 140/200
40000/40000 [==============================] - 76s 2ms/step - loss: 0.0400 - val_loss: 0.0400 - lr: 3.0157e-04
Epoch 141/200


40000/40000 [==============================] - 76s 2ms/step - loss: 0.0399 - val_loss: 0.0399 - lr: 9.3434e-05
Epoch 199/200
40000/40000 [==============================] - 76s 2ms/step - loss: 0.0399 - val_loss: 0.0399 - lr: 9.1566e-05
Epoch 200/200
40000/40000 [==============================] - 75s 2ms/step - loss: 0.0399 - val_loss: 0.0399 - lr: 8.9734e-05
Training layer 4/4
Epoch 1/200
40000/40000 [==============================] - 79s 2ms/step - loss: 0.0794 - val_loss: 0.0692 - lr: 0.0050
Epoch 2/200
40000/40000 [==============================] - 79s 2ms/step - loss: 0.0653 - val_loss: 0.0622 - lr: 0.0049
Epoch 3/200
40000/40000 [==============================] - 79s 2ms/step - loss: 0.0598 - val_loss: 0.0576 - lr: 0.0048
Epoch 4/200
40000/40000 [==============================] - 78s 2ms/step - loss: 0.0560 - val_loss: 0.0545 - lr: 0.0047
Epoch 5/200
40000/40000 [==============================] - 78s 2ms/step - loss: 0.0534 - val_loss: 0.0525 - lr: 0.0046
Epoch 6/200
40000/40000 [

40000/40000 [==============================] - 77s 2ms/step - loss: 0.0433 - val_loss: 0.0433 - lr: 0.0013
Epoch 67/200
40000/40000 [==============================] - 78s 2ms/step - loss: 0.0433 - val_loss: 0.0433 - lr: 0.0013
Epoch 68/200
40000/40000 [==============================] - 78s 2ms/step - loss: 0.0432 - val_loss: 0.0433 - lr: 0.0013
Epoch 69/200
40000/40000 [==============================] - 78s 2ms/step - loss: 0.0432 - val_loss: 0.0432 - lr: 0.0013
Epoch 70/200
40000/40000 [==============================] - 78s 2ms/step - loss: 0.0432 - val_loss: 0.0432 - lr: 0.0012
Epoch 71/200
40000/40000 [==============================] - 78s 2ms/step - loss: 0.0432 - val_loss: 0.0432 - lr: 0.0012
Epoch 72/200
40000/40000 [==============================] - 80s 2ms/step - loss: 0.0432 - val_loss: 0.0432 - lr: 0.0012
Epoch 73/200
40000/40000 [==============================] - 77s 2ms/step - loss: 0.0432 - val_loss: 0.0432 - lr: 0.0012
Epoch 74/200
40000/40000 [===========================

40000/40000 [==============================] - 78s 2ms/step - loss: 0.0429 - val_loss: 0.0429 - lr: 3.5448e-04
Epoch 133/200
40000/40000 [==============================] - 79s 2ms/step - loss: 0.0429 - val_loss: 0.0429 - lr: 3.4739e-04
Epoch 134/200
40000/40000 [==============================] - 78s 2ms/step - loss: 0.0429 - val_loss: 0.0429 - lr: 3.4044e-04
Epoch 135/200
40000/40000 [==============================] - 78s 2ms/step - loss: 0.0429 - val_loss: 0.0429 - lr: 3.3363e-04
Epoch 136/200
40000/40000 [==============================] - 79s 2ms/step - loss: 0.0429 - val_loss: 0.0429 - lr: 3.2696e-04
Epoch 137/200
40000/40000 [==============================] - 79s 2ms/step - loss: 0.0429 - val_loss: 0.0429 - lr: 3.2042e-04
Epoch 138/200
40000/40000 [==============================] - 78s 2ms/step - loss: 0.0429 - val_loss: 0.0429 - lr: 3.1401e-04
Epoch 139/200
40000/40000 [==============================] - 80s 2ms/step - loss: 0.0429 - val_loss: 0.0429 - lr: 3.0773e-04
Epoch 140/200


40000/40000 [==============================] - 78s 2ms/step - loss: 0.0428 - val_loss: 0.0428 - lr: 9.5341e-05
Epoch 198/200
40000/40000 [==============================] - 79s 2ms/step - loss: 0.0428 - val_loss: 0.0428 - lr: 9.3434e-05
Epoch 199/200
40000/40000 [==============================] - 79s 2ms/step - loss: 0.0428 - val_loss: 0.0428 - lr: 9.1566e-05
Epoch 200/200
40000/40000 [==============================] - 79s 2ms/step - loss: 0.0428 - val_loss: 0.0428 - lr: 8.9734e-05
Pos2Vec weights loading from /home/wumphlett/personal/Chess-Bot/models/pos2vec.h5


## DeepChess

In [4]:
train, val = cb.dataset.deepchess_dataset()

In [5]:
deepchess = cb.model.train_deepchess(train, val)

Pos2Vec weights loading from /home/wumphlett/personal/Chess-Bot/models/pos2vec.h5
Epoch 1/1000
20000/20000 [==============================] - 51s 2ms/step - loss: 0.5688 - accuracy: 0.6859 - val_loss: 0.4899 - val_accuracy: 0.7510 - lr: 0.0100
Epoch 2/1000
20000/20000 [==============================] - 50s 3ms/step - loss: 0.4699 - accuracy: 0.7617 - val_loss: 0.4367 - val_accuracy: 0.7750 - lr: 0.0099
Epoch 3/1000
20000/20000 [==============================] - 50s 2ms/step - loss: 0.4202 - accuracy: 0.7901 - val_loss: 0.4120 - val_accuracy: 0.7990 - lr: 0.0098
Epoch 4/1000
20000/20000 [==============================] - 50s 3ms/step - loss: 0.3959 - accuracy: 0.8055 - val_loss: 0.3860 - val_accuracy: 0.8100 - lr: 0.0097
Epoch 5/1000
20000/20000 [==============================] - 49s 2ms/step - loss: 0.3809 - accuracy: 0.8148 - val_loss: 0.3754 - val_accuracy: 0.8130 - lr: 0.0096
Epoch 6/1000
20000/20000 [==============================] - 49s 2ms/step - loss: 0.3687 - accuracy: 0.8216 -

Epoch 51/1000
20000/20000 [==============================] - 49s 2ms/step - loss: 0.2628 - accuracy: 0.8792 - val_loss: 0.3033 - val_accuracy: 0.8580 - lr: 0.0061
Epoch 52/1000
20000/20000 [==============================] - 54s 3ms/step - loss: 0.2626 - accuracy: 0.8792 - val_loss: 0.2389 - val_accuracy: 0.8880 - lr: 0.0060
Epoch 53/1000
20000/20000 [==============================] - 50s 2ms/step - loss: 0.2619 - accuracy: 0.8795 - val_loss: 0.2892 - val_accuracy: 0.8680 - lr: 0.0059
Epoch 54/1000
20000/20000 [==============================] - 51s 3ms/step - loss: 0.2609 - accuracy: 0.8800 - val_loss: 0.2570 - val_accuracy: 0.8800 - lr: 0.0059
Epoch 55/1000
20000/20000 [==============================] - 50s 2ms/step - loss: 0.2611 - accuracy: 0.8798 - val_loss: 0.2593 - val_accuracy: 0.8730 - lr: 0.0058
Epoch 56/1000
20000/20000 [==============================] - 50s 3ms/step - loss: 0.2596 - accuracy: 0.8806 - val_loss: 0.2827 - val_accuracy: 0.8600 - lr: 0.0058
Epoch 57/1000
20000/20

20000/20000 [==============================] - 49s 2ms/step - loss: 0.2380 - accuracy: 0.8917 - val_loss: 0.2576 - val_accuracy: 0.8760 - lr: 0.0037
Epoch 102/1000
20000/20000 [==============================] - 51s 3ms/step - loss: 0.2389 - accuracy: 0.8911 - val_loss: 0.2579 - val_accuracy: 0.8810 - lr: 0.0036
Epoch 103/1000
20000/20000 [==============================] - 50s 3ms/step - loss: 0.2371 - accuracy: 0.8923 - val_loss: 0.2541 - val_accuracy: 0.8860 - lr: 0.0036
Epoch 104/1000
20000/20000 [==============================] - 50s 3ms/step - loss: 0.2365 - accuracy: 0.8923 - val_loss: 0.2979 - val_accuracy: 0.8550 - lr: 0.0036
Epoch 105/1000
20000/20000 [==============================] - 51s 3ms/step - loss: 0.2366 - accuracy: 0.8926 - val_loss: 0.2886 - val_accuracy: 0.8660 - lr: 0.0035
Epoch 106/1000
20000/20000 [==============================] - 50s 2ms/step - loss: 0.2368 - accuracy: 0.8922 - val_loss: 0.2520 - val_accuracy: 0.8920 - lr: 0.0035
Epoch 107/1000
20000/20000 [===

20000/20000 [==============================] - 50s 2ms/step - loss: 0.2233 - accuracy: 0.8991 - val_loss: 0.2585 - val_accuracy: 0.8880 - lr: 0.0022
Epoch 152/1000
20000/20000 [==============================] - 52s 3ms/step - loss: 0.2231 - accuracy: 0.8991 - val_loss: 0.2395 - val_accuracy: 0.8960 - lr: 0.0022
Epoch 153/1000
20000/20000 [==============================] - 51s 3ms/step - loss: 0.2225 - accuracy: 0.8996 - val_loss: 0.2549 - val_accuracy: 0.8800 - lr: 0.0022
Epoch 154/1000
20000/20000 [==============================] - 51s 3ms/step - loss: 0.2232 - accuracy: 0.8993 - val_loss: 0.2606 - val_accuracy: 0.8790 - lr: 0.0021
Epoch 155/1000
20000/20000 [==============================] - 50s 3ms/step - loss: 0.2229 - accuracy: 0.8992 - val_loss: 0.2397 - val_accuracy: 0.8970 - lr: 0.0021
Epoch 156/1000
20000/20000 [==============================] - 50s 3ms/step - loss: 0.2214 - accuracy: 0.9000 - val_loss: 0.2566 - val_accuracy: 0.8900 - lr: 0.0021
Epoch 157/1000
20000/20000 [===

20000/20000 [==============================] - 49s 2ms/step - loss: 0.2129 - accuracy: 0.9043 - val_loss: 0.2213 - val_accuracy: 0.9100 - lr: 0.0013
Epoch 202/1000
20000/20000 [==============================] - 52s 3ms/step - loss: 0.2130 - accuracy: 0.9043 - val_loss: 0.2725 - val_accuracy: 0.8680 - lr: 0.0013
Epoch 203/1000
20000/20000 [==============================] - 50s 3ms/step - loss: 0.2135 - accuracy: 0.9034 - val_loss: 0.2642 - val_accuracy: 0.8840 - lr: 0.0013
Epoch 204/1000
20000/20000 [==============================] - 50s 3ms/step - loss: 0.2127 - accuracy: 0.9042 - val_loss: 0.2383 - val_accuracy: 0.9030 - lr: 0.0013
Epoch 205/1000
20000/20000 [==============================] - 50s 2ms/step - loss: 0.2123 - accuracy: 0.9044 - val_loss: 0.2866 - val_accuracy: 0.8700 - lr: 0.0013
Epoch 206/1000
20000/20000 [==============================] - 52s 3ms/step - loss: 0.2129 - accuracy: 0.9042 - val_loss: 0.2213 - val_accuracy: 0.8950 - lr: 0.0013
Epoch 207/1000
20000/20000 [===

...

Epoch 789/1000
20000/20000 [==============================] - 52s 3ms/step - loss: 0.1944 - accuracy: 0.9131 - val_loss: 0.2727 - val_accuracy: 0.8780 - lr: 3.6352e-06
Epoch 790/1000
20000/20000 [==============================] - 53s 3ms/step - loss: 0.1943 - accuracy: 0.9132 - val_loss: 0.2445 - val_accuracy: 0.9000 - lr: 3.5989e-06
Epoch 791/1000
20000/20000 [==============================] - 51s 3ms/step - loss: 0.1944 - accuracy: 0.9129 - val_loss: 0.2271 - val_accuracy: 0.8980 - lr: 3.5629e-06
Epoch 792/1000
20000/20000 [==============================] - 51s 3ms/step - loss: 0.1941 - accuracy: 0.9135 - val_loss: 0.2480 - val_accuracy: 0.8750 - lr: 3.5273e-06
Epoch 793/1000
20000/20000 [==============================] - 50s 3ms/step - loss: 0.1943 - accuracy: 0.9130 - val_loss: 0.2430 - val_accuracy: 0.8890 - lr: 3.4920e-06
Epoch 794/1000
20000/20000 [==============================] - 52s 3ms/step - loss: 0.1941 - accuracy: 0.9134 - val_loss: 0.2309 - val_accuracy: 0.8950 - lr: 3.4

Epoch 838/1000
20000/20000 [==============================] - 50s 2ms/step - loss: 0.1944 - accuracy: 0.9132 - val_loss: 0.2609 - val_accuracy: 0.8820 - lr: 2.2216e-06
Epoch 839/1000
20000/20000 [==============================] - 54s 3ms/step - loss: 0.1945 - accuracy: 0.9130 - val_loss: 0.2506 - val_accuracy: 0.8960 - lr: 2.1993e-06
Epoch 840/1000
20000/20000 [==============================] - 52s 3ms/step - loss: 0.1943 - accuracy: 0.9131 - val_loss: 0.2486 - val_accuracy: 0.8840 - lr: 2.1773e-06
Epoch 841/1000
20000/20000 [==============================] - 51s 3ms/step - loss: 0.1946 - accuracy: 0.9129 - val_loss: 0.2370 - val_accuracy: 0.8940 - lr: 2.1556e-06
Epoch 842/1000
20000/20000 [==============================] - 51s 3ms/step - loss: 0.1943 - accuracy: 0.9131 - val_loss: 0.2365 - val_accuracy: 0.8960 - lr: 2.1340e-06
Epoch 843/1000
20000/20000 [==============================] - 50s 2ms/step - loss: 0.1944 - accuracy: 0.9132 - val_loss: 0.2206 - val_accuracy: 0.9010 - lr: 2.1

Epoch 887/1000
20000/20000 [==============================] - 51s 3ms/step - loss: 0.1949 - accuracy: 0.9126 - val_loss: 0.2164 - val_accuracy: 0.9050 - lr: 1.3576e-06
Epoch 888/1000
20000/20000 [==============================] - 53s 3ms/step - loss: 0.1945 - accuracy: 0.9131 - val_loss: 0.2258 - val_accuracy: 0.9090 - lr: 1.3441e-06
Epoch 889/1000
20000/20000 [==============================] - 50s 3ms/step - loss: 0.1955 - accuracy: 0.9122 - val_loss: 0.2425 - val_accuracy: 0.8900 - lr: 1.3306e-06
Epoch 890/1000
20000/20000 [==============================] - 50s 3ms/step - loss: 0.1942 - accuracy: 0.9132 - val_loss: 0.2371 - val_accuracy: 0.9040 - lr: 1.3173e-06
Epoch 891/1000
20000/20000 [==============================] - 51s 3ms/step - loss: 0.1945 - accuracy: 0.9131 - val_loss: 0.2828 - val_accuracy: 0.8820 - lr: 1.3041e-06
Epoch 892/1000
20000/20000 [==============================] - 50s 3ms/step - loss: 0.1951 - accuracy: 0.9127 - val_loss: 0.2827 - val_accuracy: 0.8780 - lr: 1.2

Epoch 936/1000
20000/20000 [==============================] - 51s 3ms/step - loss: 0.1946 - accuracy: 0.9130 - val_loss: 0.2350 - val_accuracy: 0.8880 - lr: 8.2967e-07
Epoch 937/1000
20000/20000 [==============================] - 50s 3ms/step - loss: 0.1947 - accuracy: 0.9127 - val_loss: 0.2056 - val_accuracy: 0.9050 - lr: 8.2138e-07
Epoch 938/1000
20000/20000 [==============================] - 50s 2ms/step - loss: 0.1947 - accuracy: 0.9130 - val_loss: 0.2962 - val_accuracy: 0.8550 - lr: 8.1316e-07
Epoch 939/1000
20000/20000 [==============================] - 55s 3ms/step - loss: 0.1946 - accuracy: 0.9130 - val_loss: 0.2289 - val_accuracy: 0.8960 - lr: 8.0503e-07
Epoch 940/1000
20000/20000 [==============================] - 51s 3ms/step - loss: 0.1944 - accuracy: 0.9131 - val_loss: 0.2398 - val_accuracy: 0.8920 - lr: 7.9698e-07
Epoch 941/1000
20000/20000 [==============================] - 51s 3ms/step - loss: 0.1945 - accuracy: 0.9133 - val_loss: 0.2384 - val_accuracy: 0.8950 - lr: 7.8

Epoch 985/1000
20000/20000 [==============================] - 51s 3ms/step - loss: 0.1946 - accuracy: 0.9132 - val_loss: 0.2424 - val_accuracy: 0.8900 - lr: 5.0703e-07
Epoch 986/1000
20000/20000 [==============================] - 51s 3ms/step - loss: 0.1948 - accuracy: 0.9127 - val_loss: 0.2562 - val_accuracy: 0.8770 - lr: 5.0196e-07
Epoch 987/1000
20000/20000 [==============================] - 50s 2ms/step - loss: 0.1946 - accuracy: 0.9128 - val_loss: 0.2714 - val_accuracy: 0.8860 - lr: 4.9694e-07
Epoch 988/1000
20000/20000 [==============================] - 53s 3ms/step - loss: 0.1944 - accuracy: 0.9134 - val_loss: 0.2287 - val_accuracy: 0.8990 - lr: 4.9197e-07
Epoch 989/1000
20000/20000 [==============================] - 51s 3ms/step - loss: 0.1947 - accuracy: 0.9131 - val_loss: 0.2286 - val_accuracy: 0.8980 - lr: 4.8705e-07
Epoch 990/1000
20000/20000 [==============================] - 51s 3ms/step - loss: 0.1948 - accuracy: 0.9129 - val_loss: 0.2351 - val_accuracy: 0.8960 - lr: 4.8

## Evaluation

In [6]:
deepchess = cb.model.get_deepchess()
deepchess.evaluate(train)

DeepChess weights loading from /home/wumphlett/personal/Chess-Bot/models/deepchess.h5
20000/20000 [==============================] - 44s 2ms/step - loss: 0.1943 - accuracy: 0.9131


[0.1942845731973648, 0.9130880236625671]

In [7]:
deepchess.deepchess.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 dc_input_left (InputLayer)     [(None, 773)]        0           []                               
                                                                                                  
 dc_input_right (InputLayer)    [(None, 773)]        0           []                               
                                                                                                  
 pos2_vec (Pos2Vec)             (None, 100)          805100      ['dc_input_left[0][0]',          
                                                                  'dc_input_right[0][0]']         
                                                                                                  
 concatenate (Concatenate)      (None, 200)          0           ['pos2_vec[0][0]',           

In [11]:
deepchess.deepchess.layers[2].encoder.summary()

Model: "pos2vec"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 p2v_1 (Dense)               (None, 600)               464400    
                                                                 
 p2v_2 (Dense)               (None, 400)               240400    
                                                                 
 p2v_3 (Dense)               (None, 200)               80200     
                                                                 
 p2v_4 (Dense)               (None, 100)               20100     
                                                                 
Total params: 805,100
Trainable params: 805,100
Non-trainable params: 0
_________________________________________________________________


In [12]:
pos2vec.encoder.summary()

NameError: name 'pos2vec' is not defined

## Sandbox

In [ ]:
import numpy as np

train, val = cb.dataset.pos2vec_dataset()

enc = np.reshape(train[0][0][0], (1, 773))
print(pos2vec.predict(enc)[0])